<a href="https://colab.research.google.com/github/seanyx/Planet-API-Guide/blob/main/Access%20Planet%20data%20using%20APIs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The goal of this script is to develop functions in python using Planet APIs. These functions allows the users to 
1. query Planet images based on locations, time, cloud cover, and image processing level;
2. download the images resulted from the query;
3. deliver the images resulted from the query to Google Earth Engine.

To be able to run this script, you will need your Planet API Key.
And to be able to run step 3, you will need access to Google Earth Engine.

In [6]:
import os
import json
import requests
from shapely.geometry import shape, mapping

def fully_cover_geometry(aoi):
  
  from shapely.geometry import shape, mapping
  lon = [i[0] for i in list(shape(aoi).exterior.coords)]
  lat = [i[1] for i in list(shape(aoi).exterior.coords)]
  lon_range = [min(lon), max(lon)]
  lat_range = [min(lat), max(lat)]

  geometry_filter_list = [{"type": "GeometryFilter", "field_name": "geometry", "config": {"type": "Point", "coordinates": [i, j]}} for i in lon_range for j in lat_range]

  # Setup an "AND" logical filter
  and_filter = {
      "type": "AndFilter",
      "config": geometry_filter_list
  }

  return(and_filter)

def query_planet(geometry, daterange, cloudrange, item_types, fully_covered = False):

  ## planet data API link
  URL = "https://api.planet.com/data/v1"
  ## planet data API quick-search endpoint
  quick_url = "{}/quick-search".format(URL)

  # Setup the session
  session = requests.Session()
  # Authenticate
  session.auth = (PLANET_API_KEY, "")

  date_filter = {
    "type": "DateRangeFilter", # Type of filter -> Date Range
    "field_name": "acquired", # The field to filter on: "acquired" -> Date on which the "image was taken"
    "config": daterange
  }
  
  if fully_covered:
    location_filter = fully_cover_geometry(aoi)
  else: 
    location_filter = {
      "type": "GeometryFilter",
      "field_name": "geometry",
      "config": geometry
    }


  cloud_filter = {
      "type": "RangeFilter",
      "field_name": "cloud_cover",
      "config": cloudrange
  }

  # Setup an "AND" logical filter
  and_filter = {
      "type": "AndFilter",
      "config": [location_filter, date_filter, cloud_filter]
  }

  # Setup the request
  # Specify the sensors/satellites or "item types" to include in our results

  request = {
      "item_types" : item_types,
      # "interval" : "week", ## interval only required for status endpoint
      "filter" : and_filter
  }

  # Send the POST request to the API stats endpoint
  res=session.post(quick_url, json=request)

  return [i['id'] for i in res.json()["features"]]

def place_order(request, auth):

    orders_url = 'https://api.planet.com/compute/ops/orders/v2'
    response = requests.post(orders_url, data=json.dumps(request), auth=auth, headers=headers)
    print(response)
    
    if not response.ok:
        raise Exception(response.content)

    order_id = response.json()['id']
    print(order_id)
    order_url = orders_url + '/' + order_id
    return order_url

def download_zip(ids, aoi, ordername = "default", item_type = "PSScene", product_bundle = "visual"):
  
  request_zip = {
      "name": ordername,
      "products": [{
          "item_ids": ids,
          "item_type": item_type,
          "product_bundle": product_bundle
          }],
          "tools": [{
              "clip": {
                  "aoi": aoi
                  }
                  }],
          "notifications": {"email": True},
          "delivery": {"single_archive": True, "archive_type": "zip"}}
  
  order_url = place_order(request_zip, auth)

  return order_url

def deliver_to_gee(ids, aoi, gee_cloud_project_name, gee_imagecollection_name, ordername = "default", item_type = "PSScene", product_bundle = "visual"):
  
  request_gee = {
      "name": ordername,
      "products": [{
          "item_ids": ids,
          "item_type": item_type,
          "product_bundle": product_bundle
          }],
          "tools": [{
              "clip": {
                  "aoi": aoi
                  }
                  }],
          "notifications": {"email": True},
          "delivery": {
              "google_earth_engine": {
                  "project": gee_cloud_project_name,
                  "collection": gee_imagecollection_name,
                  "notifications": {
                      "email": True
                  }
              }
        }}
  
  order_url = place_order(request_gee, auth)

  return order_url

['20211230_174820_33_2414', '20211230_165427_74_242a', '20211228_170955_34_227e', '20211228_165341_35_2459', '20211222_171620_42_222f', '20211220_170039_43_241e', '20211218_174351_21_2416', '20211205_165916_66_2439', '20211203_165744_05_2463', '20211128_165902_49_2448', '20211123_174826_91_2414', '20211118_171339_16_227e', '20211117_165749_25_106c', '20211117_165429_58_2455', '20211112_165626_66_2448', '20211109_165639_51_2459', '20211107_165959_63_2458', '20211106_171511_83_222f', '20211104_174520_04_2403', '20211027_170108_75_106c', '20211025_174809_67_240a', '20211022_175045_87_2406', '20211022_165734_64_2206', '20211016_170047_82_245d', '20211013_170004_25_2423', '20211007_170020_43_241f', '20211005_170034_34_242b', '20211002_174454_99_227a', '20210928_170105_19_2429', '20210924_165828_51_242d', '20210924_174929_88_2402', '20210920_174906_87_2406', '20210915_170101_96_2436', '20210915_165741_65_2212', '20210908_174945_82_227b', '20210909_170030_85_2457', '20210906_174732_79_2403', 

Use the above functions to query Planet images

In [8]:
PLANET_API_KEY = ''
auth = (PLANET_API_KEY, "")
headers = {'content-type': 'application/json'}

# specify input below: I use https://geojson.io/#map=13/35.9204/-79.0578 to get the geojson format of an area of interest
aoi = {
        "type": "Polygon",
        "coordinates": [
          [
            [
              -79.08147811889648,
              35.897393965545646
            ],
            [
              -79.03083801269531,
              35.897393965545646
            ],
            [
              -79.03083801269531,
              35.928675705779135
            ],
            [
              -79.08147811889648,
              35.928675705779135
            ],
            [
              -79.08147811889648,
              35.897393965545646
            ]
          ]
        ]
      }

cloudrange = {
    "lte": 0.3
    }

daterange = {
    "gte": "2021-12-01T00:00:00.000Z", # "gte" -> Greater than or equal to
    "lte": "2021-12-31T00:00:00.000Z", # "gte" -> Greater than or equal to
    }

item_types = ["PSScene"]


ids = query_planet(aoi, daterange, cloudrange, item_types, True)
print(ids)
print(str(len(ids)) + ' images to be downloaded')

['20211228_155807_67_227a', '20211228_155805_37_227a', '20211227_160049_56_2402', '20211215_163201_73_1064', '20211214_155654_21_227b', '20211213_152231_13_2251', '20211212_163205_70_1064', '20211206_151027_99_2453', '20211206_152452_64_227e', '20211205_150739_24_245d', '20211203_151017_52_2460', '20211201_150730_53_2420']
12 images to be downloaded


Order images to be zipped and downloaded to local computers.

In [ ]:
order_url = download_zip(ids, aoi, ordername = "Hourse_tooth_reservoir_2021", item_type = "PSScene", product_bundle = "visual")

print(order_url)

Order images to be delivered to an image collection on Google Earth Engine cloud asset.

In [ ]:
gee_order_url = deliver_to_gee(ids, aoi, 'gee-planet-ingestion', 'planet_data_ingestion_test', ordername = "default", item_type = "PSScene", product_bundle = "visual")

print(gee_order_url)